# Report Basics of Mobile Robotics - 2024

## Table of Contents
0. [Introduction](#Introduction)
1. [Setup](#Partie-1-:-Setup)
    - 1.1 : Physical setup
    - 1.2 : Code setup
2. [Initial tuning](#Partie-2-:-Initial-tuning)
    - 2.1 : Wheel speed (differential)
3. [Vision](#Partie-2-:-Vision)
    - 3.1 : Dynamic lighting adaptation
    - 3.2 : Aruko markers
        - 3.2.1 : Thymio and goal
    - 3.2 : Map resizing
4. [Kalmann](#Partie-3-:-Kalmann)
    - 4.1 : Theory
    - 4.2 : Implementation
5. [Global path planning](#Partie-4-:-Global-path-planning)
    - 5.1 : Dijkstra
6. [Local navigation](#Partie-5-:-Local-navigation)
    - 6.1 : Local obstacle detection
    - 6.2 : Potential fields
7. [Motion control](#Partie-5-:-Motion-control)
    - 7.1 : Differential drive
    - 7.2 : P-conntroler
8. [Demonstrations](#Partie-5-:-emonstrations)
9. [Conclusion](#IConclusion)
---



## Introduction
Authors : Tifaine Mezencev, Julien, Zhuoran, Christy

---



## 1 : Setup

### 1.1 : Physical setup

```Explication des données utilisées.```

A small but determined vessel navigating uncharted waters, its sights set on a glittering treasure at the far edge of the map. Along the way, it must skillfully avoid perilous islands and evade the watchful patrols of navy ships.

In this project, we implement global and local navigation for the Thymio robot. The goal is to enable the robot to plot an optimal course toward its destination while dynamically avoiding both static and unexpected obstacles. The key features of our implementation include:

- Accurate map creation and feature localization using ArUco markers
- Global pathfinding using Dijkstra's algorithm
- Seamless connection to the Thymio robot for real-time path execution
- Path-following with dynamic adjustment for precision navigation
- Emergency handling, including "Thymio kidnapping" detection and recovery
- Local obstacle avoidance with a potential field method
- Robust navigation supported by a Kalman filter in the absence of camera input

With this project, we aim to demonstrate how a small robot can overcome big challenges, transforming obstacles into opportunities and bringing our vision of autonomous navigation to life!





### 1.2 : Code setup

## 2 : Initial tuning

### 2.1 Wheel speed (differential)

## 3: Vision

The Vision.py module has two tasks : run the camera and analyse the frames captured by the camera. This is why the Vision class is actually more of a wrapper around the functionnalities given by the two other classes defined in the vision file, appropriately named Analysis and CameraFeed. 


Let's dive into Analysis.
 
Given an image, the class has three tasks : pinpoint the thymio, the goal, and the 2D (black) obstacle in a consistent reference frame.  Of course doing this is easier said than done. A getter function of Vision will then be used to access these values and pass them on to the other modules.
The first step is defining what is in the map and what is outside the map.


### 3.1 Map

We first wanted to use color to differenciate the components of our problem and we painted our map in blue and obstacles in black. Specifically, we tried using a blue color mask to make out the map and pinpoint its four corners. This was relavitely inconsistent however, and we decided to use ArUco markers to make our life easier.

### 3.1 : Dynamic lighting adaptation

### 3.2 : ArUco markers
ArUco markers are binary patterns which are unique and have no symmetry. Due to this, they are often used in robotics and computer vision because seeing one markers makes it possible to completely determine the orientation of the camera relative to the marker (and its distance, if the size of the marker is known). They are part of a very convenient library which implements most of the code needed to detect the markers and the according translation and rotation transform matrices.

We have 6 markers : one per map corners, and two for the goal and thymio.

We wrote a function to find the center point and 2D orientation (defineed as angle between marker top edge and image X-axis). 
Yes, the aruco library already implements a rotation transform function, yes we forgot to read the docs before starting. 

#### 3.2.1 : Thymio and goal

### 3.2 : Map resizing

### 3.7 Camera management 
We want the camera to always be running in the background and display the video feed continously (along with whichever debugging displays), independently of what the path planner, control or kalman modules are doing. 

This is why we use the ```threading``` library, which allows different processes to be run in parallel. The CameraFeed class inherits from the ```threading```.Thread class. 
Its core method is a while loop that captures a frame, finds its corners, resizes it, detects the Thyimio & goal and (if desired) draws and shows the debugging displays. 

At the start of the code, the begin() method of the Vision class initializes and runs a CameraFeed thread object, which will continue its work in the background for the rest of the program.

## 4 : Kalmann

### 4.1 : Theory

### 4.2 : Implementation

## 5 : Global path planning
 


### 5.1 : Dijkstra

## 6 : Local navigation

### 6.1 : Local obstacle detection

### 6.2 : Potential fields

## 7 : Motion control

### 7.1 : Differential drive

### 7.2 : P-conntroler

## 8 : Demonstrations

---
## 9 : Conclusion
---

